In [16]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [17]:
device = torch.device("mps")
print(f"Using {device} device")

Using mps device


In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    
    def forward(self, x):
        x=self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [19]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [20]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted calss: {y_pred}")

Predicted calss: tensor([9], device='mps:0')


## 모델 계층(Layer)

In [21]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [22]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [23]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [24]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0850,  0.0716,  0.8644,  0.1016,  0.1956, -0.1166,  0.2986, -0.0334,
         -0.7047,  0.0109,  0.1498, -0.1014, -0.0188, -0.1049, -0.1143, -0.7657,
         -0.0603,  0.1711, -0.3306, -0.2240],
        [-0.1039, -0.2278,  0.5684,  0.2471,  0.5725, -0.2058,  0.4480,  0.0860,
         -0.2564, -0.0943, -0.3030, -0.3264, -0.3167, -0.0754, -0.1621, -0.5200,
         -0.3081,  0.2484, -0.3671,  0.0249],
        [ 0.2399, -0.0474,  0.7226,  0.4053,  0.2762,  0.0609,  0.1400, -0.1638,
         -0.3471, -0.0784, -0.4530,  0.0443, -0.4874, -0.0017,  0.0506, -0.4833,
         -0.0207,  0.0133, -0.5810, -0.2190]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0716, 0.8644, 0.1016, 0.1956, 0.0000, 0.2986, 0.0000, 0.0000,
         0.0109, 0.1498, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1711,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.5684, 0.2471, 0.5725, 0.0000, 0.4480, 0.0860, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

In [26]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
ligits = seq_modules(input_image)

In [27]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(ligits)

In [28]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)




/Users/chance/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1666646991213/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0258, -0.0205, -0.0079,  ...,  0.0064, -0.0281,  0.0351],
        [ 0.0114,  0.0158,  0.0033,  ..., -0.0158, -0.0168, -0.0192]],
       device='mps:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0291, -0.0059], device='mps:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-3.4180e-02,  3.8126e-02,  3.5748e-02,  ...,  2.7090e-02,
          4.0303e-02, -5.9802e-03],
        [-3.8016e-02, -3.6192e-02, -1.5214e-03,  ..., -8.1286e-05,
         -6.6796e-03,  1.9561e-03]], device='mps:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([-0.0091, -0.0369], device='mps:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[ 0.0092, -0.0279, -0.0436,  ...,  0.0279, -0.